In [1]:
import numpy as np
import pandas as pd
from nltk import tokenize, stem, corpus, download
import regex as re


In [2]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()


df = pd.read_csv("Train_zsoft.csv")
Y = encoder.fit_transform(list(df["label"]))

In [3]:
X_raw = list(df["text"])

In [4]:
def preprocessLSTM(text):
    text = text.lower()
    dateReg = r"((\d{4}.+?\d{4})|\b\d{4}\b)"
    moneyReg = r"(\d+[\., ]*)+(k|K|m|M)?($|£|€)"#(\d+\W*)+(k|K|m|M)?($|£|€)


    stopwords = corpus.stopwords.words('french') 
    stemmer = stem.SnowballStemmer("french")

    text = re.sub(moneyReg, " dollar ", text)
    text = re.sub(dateReg, " span ", text)
    text = ' '.join(stemmer.stem(word) for word in text.split() if word not in stopwords and word.isalpha())
    #okens = text.split()

    
    
    return text #" ".join([stemmer.stem(token) for token in tokens if token not in stopwords and token.isalpha()])

In [5]:
X_LSTM = []
for x in X_raw:
    X_LSTM = X_LSTM + [preprocessLSTM(x)]
for i in list(pd.read_csv("Test_zsoft.csv")["text"]):
    X_LSTM = X_LSTM + [preprocessLSTM(x)]

In [6]:
MAX_SEQUENCE_LENGTH = len(max(X_LSTM, key= lambda x : len(x)).split())

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping


2023-02-25 09:38:32.048730: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-25 09:38:32.048814: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 1300
# Max number of words in each complaint.
# This is fixed.
EMBEDDING_DIM = 50

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(X_LSTM)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 11769 unique tokens.


In [9]:
X_final = tokenizer.texts_to_matrix(X_LSTM, "binary")


In [10]:
X_final.shape

(37662, 1300)

In [11]:
#X_final = tokenizer.texts_to_sequences(X_LSTM)#list of integers
#X_final = pad_sequences(X_final, maxlen=MAX_SEQUENCE_LENGTH) #padding
X_Kaggle =  X_final[len(Y) : ]
X_final = X_final[:len(Y)]
YOneHot = pd.get_dummies(Y).values

In [12]:
# from imblearn.under_sampling import NearMiss
# undersample = NearMiss(version=2, n_neighbors=3)
# X_final_balanced, YOneHot_balanced = undersample.fit_resample(X_final, YOneHot)

In [13]:
X_train_LSTM, X_test_LSTM, Y_train_LSTM, Y_test_LSTM = train_test_split(X_final, YOneHot, test_size=0.3, random_state = 123)
print(X_train_LSTM.shape,Y_train_LSTM.shape)
print(X_test_LSTM.shape,Y_test_LSTM.shape)

(21090, 1300) (21090, 3)
(9039, 1300) (9039, 3)


In [14]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train_LSTM.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', metrics=['accuracy'])
print(model.summary())

2023-02-25 09:38:37.113056: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-25 09:38:37.113120: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-25 09:38:37.113172: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (schnoz-ThinkPad-X260): /proc/driver/nvidia/version does not exist
2023-02-25 09:38:37.113608: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1300, 50)          65000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 1300, 50)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 3)                 303       
                                                                 
Total params: 125,703
Trainable params: 125,703
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
epochs = 5
batch_size = 64
# Code Here
history = model.fit(x= X_final,
                    y = YOneHot,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[
                        EarlyStopping(monitor='val_loss',
                                      patience=3,
                                      min_delta=0.0001)
                    ])

Epoch 1/5
 31/424 [=>............................] - ETA: 15:41 - loss: 1.0250 - accuracy: 0.5277

In [ ]:
score = model.evaluate(X_final, YOneHot, verbose=0)
print('Test set\n  Loss: {:0.2f}\n  Accuracy: {:0.2f}'.format(score[0],score[1]))

Test set
  Loss: 0.46
  Accuracy: 0.80
